# Cryptohack 
## [ELLIPTIC CURVES](https://cryptohack.org/courses/elliptic/course_details/)

Writeup by Jupyter notebook

<img src="img/スクリーンショット 2023-05-04 14.13.32.png" alt="official">
crypto{Abelian}

任意の点 P に関して、 P + O = O + P = P

O 以外の2点 P(x_p, y_p), Q(x_q, y_q) に関して、 x_p = x_q かつ y_p = -y_q のとき、 P + Q = O

```
print(-6936%9739)
# 2803
```
crypto{8045,2803}

Algorithm for the addition of two points: P + Q

(a) If P = O, then P + Q = Q.<br>
(b) Otherwise, if Q = O, then P + Q = P.<br>
(c) Otherwise, write P = (x1, y1) and Q = (x2, y2).<br>
(d) If x1 = x2 and y1 = −y2, then P + Q = O.<br>
(e) Otherwise:<br>
&emsp;(e1) if P ≠ Q: λ = (y2 - y1) / (x2 - x1)<br>
&emsp;(e2) if P = Q: λ = (3x12 + a) / 2y1<br>
(f) x3 = λ2 − x1 − x2,     y3 = λ(x1 −x3) − y1<br>
(g) P + Q = (x3, y3)<br>


In [25]:
def ec_add(p, a, P, Q):
    zero = [0, 0]

    if P[0] == Q[0] and P[1] == -Q[1] % p:
        return zero
    elif P == zero:
        return Q
    elif Q == zero:
        return P
    elif Q == P:
        lamb = (pow((2*P[1]), -1, p) * (3*(P[0]**2) + a)) % p
    else:
        lamb = (pow(Q[0]-P[0], -1, p)*(Q[1]-P[1])) % p
    x = (lamb**2 - P[0] - Q[0]) % p
    # Q = (lamb**2 - Q1 - Q2) % p
    return [x, (lamb*(P[0]-x) - P[1]) % p]


p = 9739
a = 497
b = 1768
P = [493, 5564]
Q = [1539, 4742]
R = [4403, 5202]
print(ec_add(p, a, ec_add(p, a, P, P), ec_add(p, a, Q, R)))


[4215, 2162]


Double and Add algorithm for the scalar multiplication of point P by n

Input: P in E(Fp) and an integer n > 0<br>
1. Set Q = P and R = O.<br>
2. Loop while n > 0.<br>
&emsp;3. If n ≡ 1 mod 2, set R = R + Q.<br>
&emsp;4. Set Q = 2 Q and n = ⌊n/2⌋.<br>
&emsp;5. If n > 0, continue with loop at Step 2.<br>
6. Return the point R, which equals nP.<br>

In [26]:
import copy


def ec_scalar(p, a, P, n):
    Q = copy.deepcopy(P)
    R = [0, 0]
    while n > 0:
        if n % 2 == 1:
            R = ec_add(p, a, R, Q)
        Q = ec_add(p, a, Q, Q)
        n = n // 2
    return R


p = 9739
a = 497
b = 1768
n = 7863
P = [2339, 2213]
print(ec_scalar(p, a, P, n))


[9467, 2742]


In [27]:
from math import gcd
import hashlib


def legendre_symbol(a, p):
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1


def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i, 2, p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])
    return square_list


def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False


def ec(p, a, b, x):
    roots = SquareRoots(p)
    y = checkRoot(roots, (x**3+a*x+b) % p)
    return y


p = 9739
a = 497
b = 1768
G = [1804, 5368]
QA = [815, 3190]
nB = 1829

sha1 = hashlib.sha1()
data = str(ec_scalar(p, a, QA, nB)[0])
sha1.update(data.encode("utf-8"))
print(sha1.hexdigest())


80e5212754a824d3a4aed185ace4f9cac0f908bf


In [28]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib


def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))


def decrypt_flag(shared_secret: int, iv: str, ciphertext: str):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii', 'ignore'))
    key = sha1.digest()[:16]
    # Decrypt flag
    ciphertext = bytes.fromhex(ciphertext)
    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)

    if is_pkcs7_padded(plaintext):
        return unpad(plaintext, 16).decode('ascii', 'ignore')
    else:
        return plaintext.decode('ascii', 'ignore')


def ec(p, a, b, x):
    roots = SquareRoots(p)
    y = checkRoot(roots, (x**3+a*x+b) % p)
    return y

p = 9739
a = 497
b = 1768
G = [1804, 5368]
q_x = 4726
nB = 6534

data = str(ec_scalar(p, a, [q_x, ec(p, a, b, q_x)], nB)[0])
iv = 'cd9da9f1c60925922377ea952afc212c'
ciphertext = 'febcbe3a3414a730b125931dccf912d2239f3e969c4334d95ed0ec86f6449ad8'

print(decrypt_flag(data, iv, ciphertext))


crypto{3ff1c1ent_k3y_3xch4ng3}


In [34]:
# requaier y 
a = 486662
b = 1
p = 2**255 - 19
pos_Qy = []
Gx = 9

for y in range(1, p):
    if (pow(y, 2) - Gx**3 + a*(Gx**2) + Gx)%p== 0:
        pos_Qy.append(y)

print(pos_Qy)


KeyboardInterrupt: 

In [29]:
# BY^2 = X^3 + AX^2 + X (Mod p)
# Montgomery curve
import collections
def prime_factorize(n):
    a = []
    while n % 2 == 0:
        a.append(2)
        n //= 2
    f = 3
    while f * f <= n:
        if n % f == 0:
            a.append(f)
            n //= f
        else:
            f += 2
    if n != 1:
        a.append(n)
    return a

def MontgomeryCurve(p, a, b, x):
    y = (pow(b, -1, p)*(x**3 + a*(x**2) + x) % p ) % p
    print(y)
    print(collections.Counter(prime_factorize(y)))
    if legendre_symbol(p, y):
        return pow(y, (p-1)//2, p)
    return False

def MontgomeryCurve_scalar(p, a, b, k, P):
    k = bin(k)[2:]
    print(k)
    l = len(k)
    print(k)
    R[0], R[1] = P, MontgomeryCurve_double(p, a, P, P)
    for i in range(l-2):
        if i == 0:
            R[0], R[1] = MontgomeryCurve_double(
                p, a, b, R[0]), MontgomeryCurve_add(p, a, R[0], R[1])
        else:
            R[0], R[1] = MontgomeryCurve_add(
                p, a, R[0], R[1]), MontgomeryCurve_double(p, a, b, R[1], R[1])
    return R[0]


def MontgomeryCurve_add(p, a, b, P, Q):
    if P != Q:
        lamb = (pow(Q[0]-P[0], -1, p)*(Q[1]-P[1])) % p
        x = (b*(lamb**2)-a-P[0]-Q[1]) % p
        return [x, (lamb(P[0]-x)-P[1]) % p]
    else:
        return False


def MontgomeryCurve_double(p, a, b, P, Q):
    if P == Q:
        lamb = (pow(2*b*P[1], -1, p)*(3*(P[0]**2)+2*a*P[0]+1)) % p
        x = b*(lamb**2)-a-2*P[0]
        return [x, lamb(P[0]-x)-P[1]]
    return False


a = 486662
b = 1
p = 2**255 - 19
Gx = 9

print(p)
# p = 3 mod 4はメルセンヌ数（2^n-1の素数）なので、aの平方根はpow(a,(p+1)//4,p)に従う。
print()
print(2, MontgomeryCurve(p, a, b, Gx))


57896044618658097711785492504343953926634992332820282019728792003956564819949

39420360
Counter({2: 3, 3: 2, 5: 1, 7: 1, 15643: 1})
2 1


In [30]:
from Crypto.Util.number import long_to_bytes, bytes_to_long
ciphertext = b'\xd0\xe0\x83[\n\xf5%\x1b\x7f\xa0\xe9/}\x91Q\x9el)\x1c&\xe6n*\x83\xed^M\x9bt\x95\x90\x97'

m = hashlib.sha256()
m.update(long_to_bytes( dGのx座標 ))
key = m.digest()[:16]
cipher = AES.new(key, AES.MODE_ECB)
ciphertext = cipher.decrypt(pad(ciphertext, 16))
key = bytes_to_long(key)

NameError: name 'dGのx座標' is not defined

In [31]:
import random
import hashlib
from Crypto.Util.number import bytes_to_long, long_to_bytes
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from flag import flag

# secp256k1
bitsize = 256
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
a = 0
b = 7
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

d = 0xbb7bff1091c3622dab0c27ad438703b56a3be7ea763c8e6ffd1ebba7edf84427

# ref: https://ja.wikipedia.org/wiki/%E6%A5%95%E5%86%86%E6%9B%B2%E7%B7%9A%E6%9A%97%E5%8F%B7#:~:text=%E6%A5%95%E5%86%86%E6%9B%B2%E7%B7%9A%E6%9A%97%E5%8F%B7%EF%BC%88%E3%81%A0%E3%81%88%E3%82%93,%E3%81%AE%E6%A0%B9%E6%8B%A0%E3%81%A8%E3%81%99%E3%82%8B%E6%9A%97%E5%8F%B7%E3%80%82&text=%E5%85%B7%E4%BD%93%E7%9A%84%E3%81%AA%E6%9A%97%E5%8F%B7%E6%96%B9%E5%BC%8F,%E6%96%B9%E5%BC%8F%E3%81%AE%E7%B7%8F%E7%A7%B0%E3%81%A7%E3%81%82%E3%82%8B%E3%80%82
# calculate p1 + p2
def plus(p1, p2):
    if p1 == None:
        return p2
    if p2 == None:
        return p1

    x1 = p1[0]
    y1 = p1[1]
    x2 = p2[0]
    y2 = p2[1]

    phi = ((y2 - y1) * pow(x2 - x1, -1, p)) % p
    psi = ((y1*x2 - y2*x1) * pow(x2 - x1, -1, p)) % p

    x3 = (phi*phi - x1 - x2) % p
    y3 = (-phi * x3 - psi) % p
    return (x3, y3)

# calculate p1 + p1
def double(p1):
    x = p1[0]
    y = p1[1]

    phi = (((3 * x*x + a) % p) * pow(2 * y, -1, p)) % p
    psi = (((-3*x*x*x - a*x + 2*y*y) % p) * pow(2 * y, -1, p)) % p

    x4 = (phi*phi - 2 * x) % p
    y4 = (- phi * x4 - psi) % p
    return (x4, y4)

# calculate dP
def mul(d, point):
    # please implementation :)
    return None

point = (Gx, Gy)
# res = mul(d, point)

m = hashlib.sha256()
m.update(long_to_bytes(pubx))
key = m.digest()[:16]
cipher = AES.new(key, AES.MODE_ECB)
ciphertext = cipher.encrypt(pad(flag, 16))
key = bytes_to_long(key)

print('ciphertext =', ciphertext)

ModuleNotFoundError: No module named 'flag'

In [ ]:
import copy
from math import gcd

def legendre_symbol(a, p):
    """
    Computes the Legendre symbol (a|p).
    """
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1


def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i, 2, p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])
    return square_list


def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False


def ec_get_0(p, a, P):
    return ec_add(p, a, P, [P[0], -P[1] % p])


def ec_scalar(p, a, P, n):
    Q = copy.deepcopy(P)
    R = ec_get_0(p, a, P)
    while n > 0:
        if n % 2 == 1:
            R = ec_add(p, a, R, Q)
        Q = ec_add(p, a, Q, Q)
        
        n = n // 2
    return R


p = 9739
a = 497
b = 1768
n = 1337
X = [5323, 5438]
P = [2339, 2213]
# print(ec_get_o(p,a,b,P))
print(X)
print(ec_add(p, a, X, ec_get_0(p, a, X)))
print(ec_scalar(p, a, X, n))


In [ ]:
from math import gcd

# 平方剰余


def legendre_symbol(a, p):
    """
    Computes the Legendre symbol (a|p).
    """
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1

# Multiplicative Inverses in 𝔽p


def Inverses(p):
    inv_list = [0, 0]
    for i in range(p):
        for j in range(p):
            if (i*j) % p == 1:
                inv_list.append([i, j])
    return inv_list

# Square Roots Modulo N

def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i,2,p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])    
    return square_list
def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False
# 楕円曲線の足し算
def ec_add(p, squares, x1, y1, x2, y2):
    if x1 != x2:
        lamb = (y2-y1)/(x2-x1)
    else:
        lamb = (3*(x1**2) + 9)/ (2*y1)
    x = (lamb**2 - x1 - x2) % p
    # y = (lamb**2 - y1 - y2) % p
    return x, (lamb*(x1-x) - y1) % p


p = 61
x1 = 5
y1 = 7
a = 9
b = 1
x2=0
squares = SquareRoots(p)
print(ec_add(p, squares ,x1,y1,x1,y1))
print(ec_add(p, squares ,x1,y1,26,50))

while x2 < p:
    if x1 != x2:
        y2 = checkRoot(squares, (x2**3+a*x2+b)%p)
        if y2:
            x3, y3 = ec_add(p, x1, y1, x2, y2)
            print(x3, y3, x2, y2)
            if x3 == 0 and y3 == 0:
                print(x2, y2)
                break
    x2 += 1


In [ ]:
from math import gcd

# 平方剰余


def legendre_symbol(a, p):
    """
    Computes the Legendre symbol (a|p).
    """
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1

# Multiplicative Inverses in 𝔽p


def Inverses(p):
    inv_list = [0, 0]
    for i in range(p):
        for j in range(p):
            if (i*j) % p == 1:
                inv_list.append([i, j])
    return inv_list

# Square Roots Modulo N


def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i, 2, p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])
    return square_list


def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False

# 楕円曲線の足し算


def ec_add(p, x1, y1, x2, y2):
    if x1 != x2:
        lamb = (y2-y1)/(x2-x1)
    else:
        lamb = (3*(x1**2) + 9) / (2*y1)
    x = (lamb**2 - x1 - x2) % p
    # y = (lamb**2 - y1 - y2) % p
    return x, (lamb*(x1-x) - y1) % p


p = 9739
x1 = 8045
y1 = 6936
a = 497
b = 1768
x2 = 0
squares = SquareRoots(p)
x0 = 0
y0 = checkRoot(squares, (x0**3+a*x0+b) % p)
print(x0,y0)
while x2 < p:
    if x1 != x2:
        y2 = checkRoot(squares, (x2**3+a*x2+b) % p)
        if y2:
            x3, y3 = ec_add(p, x1, y1, x2, y2)
            if x3 == x0 and y3 == y0:
                print(x2, y2)
                break
    x2 += 1


In [ ]:
from math import gcd

# 平方剰余


def legendre_symbol(a, p):
    """
    Computes the Legendre symbol (a|p).
    """
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1

# Multiplicative Inverses in 𝔽p


def Inverses(p):
    inv_list = [0, 0]
    for i in range(p):
        for j in range(p):
            if (i*j) % p == 1:
                inv_list.append([i, j])
    return inv_list

# Square Roots Modulo N

def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i,2,p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])    
    return square_list
def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False
# 楕円曲線の足し算
def ec_add(p, x1, y1, x2, y2):
    lamb = (y2-y1)/(x2-x1)
    x = (lamb**2 - x1 - x2) % p
    # y = (lamb**2 - y1 - y2) % p
    return x, (lamb*(x1-x) - y1) % p


p = 61
x1 = 8
y1 = 6936
a = 497
b = 1768
a = b = x2 = 0
x2=2
squares = SquareRoots(p)
while x2 < p:
    if x1 != x2:
        y2 = checkRoot(squares, (x2**3+a*x2+b)%p)
        if y2:
            x3, y3 = ec_add(p, x1, y1, x2, y2)
            print(x3, y3, x2, y2)
            if x3 == 0 and y3 == 0:
                print(x2, y2)
                break
    x2 += 1


In [ ]:
p=93
for i in range(p):
    print(i, 3**i % 93) 